In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [13]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///resources/Chinook.db")

In [14]:
from langchain.tools import tool

@tool
def sql_query(query: str) -> str:

    """Obtain information from the database using SQL queries"""

    try:
        return db.run(query)
    except Exception as e:
        return f"Error: {e}"

sql_query.invoke("SELECT * FROM Artist LIMIT 10")

"[(1, 'AC/DC'), (2, 'Accept'), (3, 'Aerosmith'), (4, 'Alanis Morissette'), (5, 'Alice In Chains'), (6, 'Antônio Carlos Jobim'), (7, 'Apocalyptica'), (8, 'Audioslave'), (9, 'BackBeat'), (10, 'Billy Cobham')]"

In [15]:
from langchain.agents import create_agent

agent = create_agent(
    model="gpt-5-nano",
    tools=[sql_query]
)

In [16]:
from langchain.messages import HumanMessage

question = HumanMessage(content="Who is the most popular artist beginning with 'S' in this database?")

response = agent.invoke(
    {"messages": [question]}
)

In [20]:
from pprint import pprint

pprint(response['messages'])

[HumanMessage(content="Who is the most popular artist beginning with 'S' in this database?", additional_kwargs={}, response_metadata={}, id='9b99b80e-d73d-4296-a576-003b85202e74'),
 AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 744, 'prompt_tokens': 142, 'total_tokens': 886, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 704, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-D7ZCrIXVV9Ge05ewm8MX4SHX1Z4OO', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c45b4-c722-7861-8046-a203206d04bc-0', tool_calls=[{'name': 'sql_query', 'args': {'query': "SELECT name FROM artists WHERE name LIKE 'S%' ORDER BY popularity DESC LIMIT 1;"}, 'id': 'call_fGmqUoP8QV1vfmE7Nqd0jnUe',

In [21]:
# Safely get the SQL query from the first message that made a tool call
messages = response["messages"]
for msg in reversed(messages):
    tool_calls = getattr(msg, "tool_calls", None) or []
    if tool_calls and tool_calls[0].get("args", {}).get("query"):
        print(tool_calls[0]["args"]["query"])
        break
else:
    print("No SQL tool call in this response (agent may have asked a follow-up instead).")

SELECT a.Name, COUNT(*) AS TrackCount
FROM Artist a
JOIN Album al ON al.ArtistId = a.ArtistId
JOIN Track t ON t.AlbumId = al.AlbumId
WHERE a.Name LIKE 'S%'
GROUP BY a.Name
ORDER BY TrackCount DESC
LIMIT 1;
